In [1]:
import zstandard as zstd
import io
import pandas as pd
import glob
import os

In [16]:
data_d = '/home/lucytian/data/4_polypred/gwas_geno'

trait_type = {
    'INI30120': 'blood_count',
    'INI20030780': 'blood_biochemistry',
    'INI1003063': 'spirometry',
    'INI50030700': 'blood_biochemistry'
}

for pop in ['WB_train_val', 'Afr_train_val']:
    # Read afreq once per population
    afreq_path = f'/net/bmc-lab5/data/kellis/group/tanigawa/data/ukb21942/geno/afreq_HWE/population/ukb_genoHM3.{pop}.afreq.gz'
    afreq = pd.read_csv(afreq_path, sep='\t', compression='gzip')

    for i in ['INI30120', 'INI20030780', 'INI1003063', 'INI50030700']:
        dfs = []
        dir_path = f"{data_d}/{trait_type[i]}/pop_10PCs/{pop}/var_split"
        pattern = os.path.join(dir_path, f"*{i}*")
        matching_files = glob.glob(pattern)

        for file_path in matching_files:
            with open(file_path, 'rb') as fh:
                dctx = zstd.ZstdDecompressor()
                stream_reader = dctx.stream_reader(fh)
                text_stream = io.TextIOWrapper(stream_reader, encoding='utf-8')
                df = pd.read_csv(text_stream, sep='\t')
                dfs.append(df)

        if dfs:  # merge and save only if we found files
            df_full = pd.concat(dfs).sort_values(by=['#CHROM', 'POS'])
            df_full = df_full.merge(afreq[['ID', 'ALT', 'ALT_FREQS']], on=['ID', 'ALT'])
            out_path = f'{data_d}/{trait_type[i]}/pop_10PCs/{pop}/{i}.glm.tsv.gz'
            df_full.to_csv(out_path, sep='\t', index=False, compression='gzip')

/tmp/ipykernel_21211/1159573828.py:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  afreq = pd.read_csv(afreq_path, sep='\t', compression='gzip')
/tmp/ipykernel_21211/1159573828.py:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  afreq = pd.read_csv(afreq_path, sep='\t', compression='gzip')
